In [1]:
# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).


In [3]:
# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.


In [4]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data_for_gan', download = True, transform = transform) # We download the training set in the ./data folder and we apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch.


Files already downloaded and verified


In [5]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [6]:
# Defining the generator

class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

In [7]:
# Creating the generator
netG = G()
netG.apply(weights_init)

G (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU (inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU (inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU (inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU (inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh ()
  )
)

In [8]:
# Defining the discriminator

class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

In [9]:
# Creating the discriminator
netD = D()
netD.apply(weights_init)

D (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid ()
  )
)

In [10]:
# Training the DCGANs

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))


In [ ]:
for epoch in range(25):

    for i, data in enumerate(dataloader, 0):
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad()
        
        # Training the discriminator with a real image of the dataset
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)
        
        # Backpropagating the total error
        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        
        # 3rd Step: Printing the losses and saving the real images and the generated images of the minibatch every 100 steps

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.data[0], errG.data[0]))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)

[0/25][0/782] Loss_D: 2.2372 Loss_G: 5.4773
[0/25][1/782] Loss_D: 1.1970 Loss_G: 6.4851
[0/25][2/782] Loss_D: 0.8306 Loss_G: 6.0550
[0/25][3/782] Loss_D: 0.9350 Loss_G: 6.3628
[0/25][4/782] Loss_D: 0.9163 Loss_G: 6.6590
[0/25][5/782] Loss_D: 0.7395 Loss_G: 7.3583
[0/25][6/782] Loss_D: 0.5526 Loss_G: 8.3548
[0/25][7/782] Loss_D: 0.7025 Loss_G: 7.6987
[0/25][8/782] Loss_D: 1.1653 Loss_G: 9.2828
[0/25][9/782] Loss_D: 0.5147 Loss_G: 7.7485
[0/25][10/782] Loss_D: 0.9810 Loss_G: 11.3683
[0/25][11/782] Loss_D: 0.4143 Loss_G: 8.3494
[0/25][12/782] Loss_D: 1.1858 Loss_G: 12.7913
[0/25][13/782] Loss_D: 0.2679 Loss_G: 10.0830
[0/25][14/782] Loss_D: 0.5461 Loss_G: 9.7823
[0/25][15/782] Loss_D: 0.6714 Loss_G: 12.1816
[0/25][16/782] Loss_D: 0.5695 Loss_G: 9.1460
[0/25][17/782] Loss_D: 0.8320 Loss_G: 14.3816
[0/25][18/782] Loss_D: 0.2426 Loss_G: 11.4293
[0/25][19/782] Loss_D: 0.3607 Loss_G: 8.0039
[0/25][20/782] Loss_D: 1.3295 Loss_G: 17.2308
[0/25][21/782] Loss_D: 0.5859 Loss_G: 15.9229
[0/25][22/78

[0/25][180/782] Loss_D: 0.4171 Loss_G: 7.4539
[0/25][181/782] Loss_D: 0.2623 Loss_G: 6.0096
[0/25][182/782] Loss_D: 0.1637 Loss_G: 4.9508
[0/25][183/782] Loss_D: 0.4082 Loss_G: 7.1397
[0/25][184/782] Loss_D: 0.3091 Loss_G: 4.5394
[0/25][185/782] Loss_D: 0.3658 Loss_G: 7.3058
[0/25][186/782] Loss_D: 0.2665 Loss_G: 5.5245
[0/25][187/782] Loss_D: 0.2264 Loss_G: 5.7400
[0/25][188/782] Loss_D: 0.1981 Loss_G: 5.0859
[0/25][189/782] Loss_D: 0.3032 Loss_G: 6.4712
[0/25][190/782] Loss_D: 0.1920 Loss_G: 5.5285
[0/25][191/782] Loss_D: 0.1563 Loss_G: 5.3642
[0/25][192/782] Loss_D: 0.1995 Loss_G: 6.0784
[0/25][193/782] Loss_D: 0.2325 Loss_G: 4.8749
[0/25][194/782] Loss_D: 0.2217 Loss_G: 7.1798
[0/25][195/782] Loss_D: 0.1203 Loss_G: 6.3674
[0/25][196/782] Loss_D: 0.1417 Loss_G: 5.1199
[0/25][197/782] Loss_D: 0.2567 Loss_G: 7.6052
[0/25][198/782] Loss_D: 0.2706 Loss_G: 5.2882
[0/25][199/782] Loss_D: 0.1676 Loss_G: 7.0499
[0/25][200/782] Loss_D: 0.1336 Loss_G: 6.3664
[0/25][201/782] Loss_D: 0.1114 Los

[0/25][358/782] Loss_D: 2.0887 Loss_G: 16.3032
[0/25][359/782] Loss_D: 3.9616 Loss_G: 11.6658
[0/25][360/782] Loss_D: 0.3198 Loss_G: 8.2352
[0/25][361/782] Loss_D: 0.3710 Loss_G: 6.7083
[0/25][362/782] Loss_D: 0.3439 Loss_G: 7.8645
[0/25][363/782] Loss_D: 0.2225 Loss_G: 6.3732
[0/25][364/782] Loss_D: 0.3701 Loss_G: 6.4199
[0/25][365/782] Loss_D: 0.3049 Loss_G: 6.4325
[0/25][366/782] Loss_D: 0.2456 Loss_G: 5.9837
[0/25][367/782] Loss_D: 0.3825 Loss_G: 6.5897
[0/25][368/782] Loss_D: 0.4946 Loss_G: 5.5142
[0/25][369/782] Loss_D: 0.4618 Loss_G: 11.5774
[0/25][370/782] Loss_D: 0.8604 Loss_G: 8.2482
[0/25][371/782] Loss_D: 0.1391 Loss_G: 4.2777
[0/25][372/782] Loss_D: 1.3361 Loss_G: 14.0497
[0/25][373/782] Loss_D: 1.7566 Loss_G: 11.0022
[0/25][374/782] Loss_D: 0.3865 Loss_G: 6.3632
[0/25][375/782] Loss_D: 0.2946 Loss_G: 5.7419
[0/25][376/782] Loss_D: 0.3822 Loss_G: 7.7864
[0/25][377/782] Loss_D: 0.2182 Loss_G: 6.7566
[0/25][378/782] Loss_D: 0.2447 Loss_G: 5.2041
[0/25][379/782] Loss_D: 0.558

[0/25][536/782] Loss_D: 0.3950 Loss_G: 5.8273
[0/25][537/782] Loss_D: 0.3454 Loss_G: 4.8852
[0/25][538/782] Loss_D: 0.4373 Loss_G: 3.9449
[0/25][539/782] Loss_D: 0.5492 Loss_G: 5.7318
[0/25][540/782] Loss_D: 0.2892 Loss_G: 5.5115
[0/25][541/782] Loss_D: 0.3512 Loss_G: 4.9280
[0/25][542/782] Loss_D: 0.7589 Loss_G: 5.4104
[0/25][543/782] Loss_D: 0.6978 Loss_G: 4.2285
[0/25][544/782] Loss_D: 1.0928 Loss_G: 3.8645
[0/25][545/782] Loss_D: 0.4940 Loss_G: 5.0116
[0/25][546/782] Loss_D: 0.7203 Loss_G: 2.4739
[0/25][547/782] Loss_D: 1.1858 Loss_G: 7.6023
[0/25][548/782] Loss_D: 0.9450 Loss_G: 4.7479
[0/25][549/782] Loss_D: 0.3938 Loss_G: 3.8455
[0/25][550/782] Loss_D: 0.4392 Loss_G: 4.6858
[0/25][551/782] Loss_D: 0.4471 Loss_G: 3.5081
[0/25][552/782] Loss_D: 0.7638 Loss_G: 2.5977
[0/25][553/782] Loss_D: 0.8578 Loss_G: 5.8452
[0/25][554/782] Loss_D: 0.8038 Loss_G: 3.4920
[0/25][555/782] Loss_D: 0.3065 Loss_G: 3.2765
[0/25][556/782] Loss_D: 0.6530 Loss_G: 3.0711
[0/25][557/782] Loss_D: 0.8273 Los

[0/25][715/782] Loss_D: 0.1500 Loss_G: 8.0604
[0/25][716/782] Loss_D: 0.0626 Loss_G: 7.4049
[0/25][717/782] Loss_D: 0.1345 Loss_G: 5.1880
[0/25][718/782] Loss_D: 0.3648 Loss_G: 14.1223
[0/25][719/782] Loss_D: 0.7964 Loss_G: 10.0186
[0/25][720/782] Loss_D: 0.0588 Loss_G: 6.2522
[0/25][721/782] Loss_D: 0.4088 Loss_G: 11.2832
[0/25][722/782] Loss_D: 0.2145 Loss_G: 9.2792
[0/25][723/782] Loss_D: 0.3685 Loss_G: 9.2177
[0/25][724/782] Loss_D: 0.4081 Loss_G: 5.0083
[0/25][725/782] Loss_D: 1.1106 Loss_G: 17.8044
[0/25][726/782] Loss_D: 2.7034 Loss_G: 9.6176
[0/25][727/782] Loss_D: 0.1281 Loss_G: 5.6482
[0/25][728/782] Loss_D: 0.6907 Loss_G: 9.7399
[0/25][729/782] Loss_D: 0.3317 Loss_G: 7.0894
[0/25][730/782] Loss_D: 0.2707 Loss_G: 6.0329
[0/25][731/782] Loss_D: 0.3102 Loss_G: 7.9529
[0/25][732/782] Loss_D: 0.7770 Loss_G: 1.9441
[0/25][733/782] Loss_D: 1.3143 Loss_G: 11.7156
[0/25][734/782] Loss_D: 0.2169 Loss_G: 12.7913
[0/25][735/782] Loss_D: 0.7791 Loss_G: 7.7936
[0/25][736/782] Loss_D: 0.06

[1/25][114/782] Loss_D: 0.2659 Loss_G: 3.8023
[1/25][115/782] Loss_D: 0.4896 Loss_G: 1.9721
[1/25][116/782] Loss_D: 0.5339 Loss_G: 3.2086
[1/25][117/782] Loss_D: 0.3690 Loss_G: 4.6626
[1/25][118/782] Loss_D: 0.7989 Loss_G: 2.0627
[1/25][119/782] Loss_D: 0.5322 Loss_G: 3.0251
[1/25][120/782] Loss_D: 0.5519 Loss_G: 3.4662
[1/25][121/782] Loss_D: 0.5741 Loss_G: 2.6061
[1/25][122/782] Loss_D: 0.6307 Loss_G: 4.2833
[1/25][123/782] Loss_D: 0.3907 Loss_G: 3.9419
[1/25][124/782] Loss_D: 0.6224 Loss_G: 2.0816
[1/25][125/782] Loss_D: 0.9232 Loss_G: 5.8993
[1/25][126/782] Loss_D: 0.6208 Loss_G: 3.9322
[1/25][127/782] Loss_D: 0.2641 Loss_G: 3.9609
[1/25][128/782] Loss_D: 0.1433 Loss_G: 4.2077
[1/25][129/782] Loss_D: 0.3565 Loss_G: 3.5574
[1/25][130/782] Loss_D: 0.4974 Loss_G: 4.3057
[1/25][131/782] Loss_D: 0.6846 Loss_G: 2.5189
[1/25][132/782] Loss_D: 0.7441 Loss_G: 4.9922
[1/25][133/782] Loss_D: 0.8632 Loss_G: 0.7562
[1/25][134/782] Loss_D: 2.0383 Loss_G: 10.0907
[1/25][135/782] Loss_D: 1.7567 Lo

[1/25][293/782] Loss_D: 0.2213 Loss_G: 3.8747
[1/25][294/782] Loss_D: 0.3337 Loss_G: 2.3063
[1/25][295/782] Loss_D: 0.4957 Loss_G: 5.7066
[1/25][296/782] Loss_D: 0.9647 Loss_G: 1.3003
[1/25][297/782] Loss_D: 0.8707 Loss_G: 6.2273
[1/25][298/782] Loss_D: 0.3827 Loss_G: 4.5053
[1/25][299/782] Loss_D: 0.2875 Loss_G: 3.0268
[1/25][300/782] Loss_D: 0.5408 Loss_G: 4.8396
[1/25][301/782] Loss_D: 0.3990 Loss_G: 2.5820
[1/25][302/782] Loss_D: 0.3955 Loss_G: 3.1475
[1/25][303/782] Loss_D: 0.3670 Loss_G: 5.6021
[1/25][304/782] Loss_D: 0.4942 Loss_G: 2.3650
[1/25][305/782] Loss_D: 0.5632 Loss_G: 6.1681
[1/25][306/782] Loss_D: 0.5114 Loss_G: 2.7955
[1/25][307/782] Loss_D: 0.4293 Loss_G: 4.8509
[1/25][308/782] Loss_D: 0.3589 Loss_G: 3.8705
[1/25][309/782] Loss_D: 0.2009 Loss_G: 4.5469
[1/25][310/782] Loss_D: 0.1339 Loss_G: 4.6000
[1/25][311/782] Loss_D: 0.2726 Loss_G: 3.6243
[1/25][312/782] Loss_D: 0.2860 Loss_G: 3.1289
[1/25][313/782] Loss_D: 0.2968 Loss_G: 4.9931
[1/25][314/782] Loss_D: 0.2117 Los

[1/25][472/782] Loss_D: 0.6660 Loss_G: 4.1650
[1/25][473/782] Loss_D: 0.5174 Loss_G: 2.3964
[1/25][474/782] Loss_D: 0.3917 Loss_G: 3.4706
[1/25][475/782] Loss_D: 0.5448 Loss_G: 4.7261
[1/25][476/782] Loss_D: 0.9345 Loss_G: 1.5243
[1/25][477/782] Loss_D: 0.9846 Loss_G: 4.7485
[1/25][478/782] Loss_D: 0.7742 Loss_G: 2.6869
[1/25][479/782] Loss_D: 0.7180 Loss_G: 4.2355
[1/25][480/782] Loss_D: 0.5451 Loss_G: 3.0077
[1/25][481/782] Loss_D: 0.4060 Loss_G: 4.3497
[1/25][482/782] Loss_D: 0.4707 Loss_G: 3.1523
[1/25][483/782] Loss_D: 0.4641 Loss_G: 3.0924
[1/25][484/782] Loss_D: 0.6259 Loss_G: 5.4356
[1/25][485/782] Loss_D: 0.9470 Loss_G: 2.2676
[1/25][486/782] Loss_D: 0.8575 Loss_G: 4.0077
[1/25][487/782] Loss_D: 0.6334 Loss_G: 4.4567
[1/25][488/782] Loss_D: 0.9747 Loss_G: 1.9322
[1/25][489/782] Loss_D: 0.7827 Loss_G: 5.9447
[1/25][490/782] Loss_D: 0.5087 Loss_G: 3.8438
[1/25][491/782] Loss_D: 0.5534 Loss_G: 2.9061
[1/25][492/782] Loss_D: 0.4238 Loss_G: 4.7101
[1/25][493/782] Loss_D: 0.7093 Los

[1/25][651/782] Loss_D: 0.3831 Loss_G: 3.2373
[1/25][652/782] Loss_D: 0.4118 Loss_G: 4.1651
[1/25][653/782] Loss_D: 0.2344 Loss_G: 4.2699
[1/25][654/782] Loss_D: 0.3635 Loss_G: 2.7671
[1/25][655/782] Loss_D: 0.4129 Loss_G: 3.8056
[1/25][656/782] Loss_D: 0.4869 Loss_G: 4.2416
[1/25][657/782] Loss_D: 0.4219 Loss_G: 3.0658
[1/25][658/782] Loss_D: 0.3711 Loss_G: 4.4479
[1/25][659/782] Loss_D: 0.4338 Loss_G: 3.4128
[1/25][660/782] Loss_D: 0.3883 Loss_G: 3.4813
[1/25][661/782] Loss_D: 0.5184 Loss_G: 4.4233
[1/25][662/782] Loss_D: 0.4669 Loss_G: 2.3879
[1/25][663/782] Loss_D: 0.5817 Loss_G: 5.9557
[1/25][664/782] Loss_D: 0.5075 Loss_G: 3.4927
[1/25][665/782] Loss_D: 0.2561 Loss_G: 3.6444
[1/25][666/782] Loss_D: 0.5114 Loss_G: 6.3337
[1/25][667/782] Loss_D: 0.6116 Loss_G: 2.5372
[1/25][668/782] Loss_D: 0.4697 Loss_G: 4.4447
[1/25][669/782] Loss_D: 0.2946 Loss_G: 3.9596
[1/25][670/782] Loss_D: 0.5099 Loss_G: 2.4285
[1/25][671/782] Loss_D: 0.6318 Loss_G: 6.1365
[1/25][672/782] Loss_D: 0.6113 Los

[2/25][49/782] Loss_D: 0.4859 Loss_G: 3.5061
[2/25][50/782] Loss_D: 0.4954 Loss_G: 2.9230
[2/25][51/782] Loss_D: 0.3962 Loss_G: 4.1482
[2/25][52/782] Loss_D: 0.5790 Loss_G: 2.6620
[2/25][53/782] Loss_D: 0.4964 Loss_G: 5.1774
[2/25][54/782] Loss_D: 0.4136 Loss_G: 3.3630
[2/25][55/782] Loss_D: 0.3234 Loss_G: 4.4399
[2/25][56/782] Loss_D: 0.2630 Loss_G: 3.8763
[2/25][57/782] Loss_D: 0.2412 Loss_G: 3.1286
[2/25][58/782] Loss_D: 0.2924 Loss_G: 4.3223
[2/25][59/782] Loss_D: 0.5561 Loss_G: 2.9898
[2/25][60/782] Loss_D: 0.3263 Loss_G: 4.3646
[2/25][61/782] Loss_D: 0.3450 Loss_G: 2.7162
[2/25][62/782] Loss_D: 0.4023 Loss_G: 4.7259
[2/25][63/782] Loss_D: 0.3857 Loss_G: 3.0197
[2/25][64/782] Loss_D: 0.2609 Loss_G: 3.4732
[2/25][65/782] Loss_D: 0.3509 Loss_G: 3.9738
[2/25][66/782] Loss_D: 0.4973 Loss_G: 2.1280
[2/25][67/782] Loss_D: 0.5594 Loss_G: 6.1877
[2/25][68/782] Loss_D: 0.7968 Loss_G: 2.0112
[2/25][69/782] Loss_D: 0.4282 Loss_G: 4.0394
[2/25][70/782] Loss_D: 0.2163 Loss_G: 4.6534
[2/25][71/